# Task 2: Populating Company Presence on the Internet



BeautifulSoup and requests can only scrape static websites, but they cannot scrape the dynamic website that uses JS.
Dynamically loading the content using JS is also known as AJAX(Asynchronous JavaScript and XML)

Parsing-converting a string representation of a Python object into an actual object.
Rendering- interpreting HTML, CSS, JS, and images into something that we see in a browser.

 (a way to deal with pagination)

Basic steps to perform this task:
1. Connect to the MySQL database and extract company English names from the company_master table.
2. Use these names to search on Google and extract the top 10 links for each company.
3. Send an initial request to get the first page of search results.
4. Extract links from the first page.
5. Find the link to the next page and send a request to it if more results are needed.
6. Repeat the process until you have collected the desired number of links (10 in this case).
7. Update the `COMPANY_MASTER` table with the gathered links.

In [2]:
import requests
import re
from bs4 import BeautifulSoup

In [3]:

def get_google_search_results(query, num_results=10):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                      '(KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    base_url = 'https://google.com'
    search_url = f'{base_url}/search?q={query}'
    links = []

    while len(links) < num_results:
        try:
            response = requests.get(search_url, headers=headers)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            # Find all <h3> tags in the current page of search results
            for heading in soup.find_all('h3'):
                parent_tag = heading.find_parent('a')
                if parent_tag and 'href' in parent_tag.attrs:
                    link = parent_tag['href'].strip()
                    if re.match(r'^https?://(?!www\.google\.).+', link):
                        links.append(link)
                        if len(links) >= num_results:
                            break

            # Find the link to the next page
            next_page = soup.select_one('a[id="pnnext"]')
            if next_page and next_page['href']:
                search_url = base_url + next_page['href']
            else:
                break  # No more pages

        except requests.RequestException as e:
            print(f"Error fetching URL: {e}")
            break

    return links[:num_results]

text = "f1soft"
top_links = get_google_search_results(text, num_results=10)

print("Top 10 Links:")
for i, link in enumerate(top_links, 1):
    print(f"{i}: {link}")


Top 10 Links:
1: https://f1soft.com/
2: https://np.linkedin.com/company/f1softgroup
3: https://www.facebook.com/f1soft/
4: https://twitter.com/f1soft
5: https://play.google.com/store/apps/developer?id=F1soft&hl=en_US
6: https://merojob.com/employer/f1soft-international/
7: https://np.linkedin.com/in/sharmasubash
8: https://np.linkedin.com/in/sharmasubash
9: https://www.jobejee.com/employer/F1soft-International-/2280
10: https://www.talentconnects.com.np/f1soft-international-29/career
